In [130]:
import requests
from datetime import datetime
import pandas as pd

In [173]:
params = {
    'text': 'NAME:"программист 1С" OR "1C программист"', # Текст фильтра. В имени должно быть слово "Аналитик"
    'area': 73, # Петрозаводск
    'page': 0, # Индекс страницы поиска на HH
    'per_page': 100 # Кол-во вакансий на 1 странице
    }
r = requests.get('https://api.hh.ru/vacancies', params).json()

In [174]:
r['pages']

1

In [175]:
len(r['items'])

3

In [176]:
for i in range(len(r['items'])):
    print(r['items'][i]['name'])

Инженер-программист 1С
Разработчик 1С
1 С Программист


In [177]:
r['items'][0]['name']

'Инженер-программист 1С'

In [178]:
r['items'][0]['alternate_url']

'https://hh.ru/vacancy/54687609'

In [179]:
r['items'][0]['salary']['from']
r['items'][0]['salary']['to']

80000

In [189]:
r['items'][2]

{'id': '54998789',
 'premium': False,
 'name': '1 С Программист',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '73',
  'name': 'Петрозаводск',
  'url': 'https://api.hh.ru/areas/73'},
 'salary': {'from': 60000, 'to': None, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Петрозаводск',
  'street': 'район Голиковка, Повенецкая улица',
  'building': '16с2',
  'description': None,
  'lat': 61.771271,
  'lng': 34.352115,
  'raw': 'Петрозаводск, район Голиковка, Повенецкая улица, 16с2',
  'metro': None,
  'metro_stations': [],
  'id': '692085'},
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2022-04-25T12:52:02+0300',
 'created_at': '2022-04-25T12:52:02+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=54998789',
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/54998789?host=hh.ru',
 'alternate_url':